# Übung Datenbanken: Views

![](https://www.klipfel-mathys.ch/images/imagesCAUM394L.jpg)

In [ ]:
%%sh

#rm -f opendata.sql zugauskunft.db
#wget -qOopendata.sql 'https://nextcloud.th-deg.de/s/Kc7m486Z3KbZyB4/download?path=%2F&files=bahn-opendata.sql'
#cat ./opendata.sql | sqlite3 zugauskunft.db

wget -qObahn-opendata.db 'https://nextcloud.th-deg.de/s/Kc7m486Z3KbZyB4/download?path=%2F&files=bahn-opendata.db'
cp bahn-opendata.db zugauskunft.db

In [ ]:
# Hier ist nur Code zum Initialisieren der Umgebeung, bitte gehen Sie weiter, es gibt nichts zu sehen.

# Keine langen Fehlermeldungen
import sys
ipython = get_ipython()

def exception_handler(exception_type, exception, traceback):
    print("%s: %s" % (exception_type.__name__, exception), file=sys.stderr)

ipython._showtraceback = exception_handler

# Lade die Erweiterung, damit wir SQL Befehle nutzen können
%reload_ext sql
%sql sqlite:///zugauskunft.db


## Über das Importieren von Daten

Wir erinnern uns an die Aufgabenstellung aus dem Übungsblatt. Wir verwenden hier folgende Modellierung.

![](https://nextcloud.th-deg.de/s/Kc7m486Z3KbZyB4/download?path=%2F&files=bahn.png)


Eine Übung lebt ja davon, dass man ein paar Daten zum rumspielen hat. Daher habe ich Daten habe aus einem [bundesweiten Opendataportal](https://www.opendata-oepnv.de/ht/de/organisation/delfi/startseite?tx_vrrkit_view%5Bdataset_name%5D=deutschlandweite-sollfahrplandaten-gtfs&tx_vrrkit_view%5Bdataset_formats%5D%5B0%5D=ZIP&tx_vrrkit_view%5Baction%5D=details&tx_vrrkit_view%5Bcontroller%5D=View) genommen. Wir sollten damit in der Lage sein echte Verbindungen nachzuvollziehen. Ich habe den Datensatz auf die Waldbahn beschränkt. Der Datensatz selbst ist im CSV Format (!), welches aber von SQLite importiert werden kann.

Die Daten selbst habe ich Ihnen bereits gefiltert und importiert (selbst der bayernweite Datensatz ist recht sperrig), nun geht es darum einige Views zu erstellen.

Die Quellstruktur der GTFS Daten ist wie folgt abgebildet:

![](https://nextcloud.th-deg.de/s/Kc7m486Z3KbZyB4/download?path=%2F&files=gtfs-er.png)

## Aufgabe 1

#### Aufgabe 1.1
Betrachten wir erst einmal die gegebenen Daten. Wir beginnen mit der `routes` Tabelle, in der alle Routen, die von einem Betreiber (Agency) bedient werden abgelegt sind. Der vorliegende Datensatz beschränkt sich auf die Waldbahn, welche die Agency ID 10964 hat. Fragen Sie daher alle Daten aus dieser Tabelle ab.

#### Aufgabe 1.2
Die Routen selbst sind scheinbar also nicht sehr spannend. Trips sind dann die einzelnen Fahrten entlang der Routen, die gefahren werden. Also das, was man den Fahrplan nennt. Fragen Sie doch mal die ersten 10 Fahrten der WBA2 ab.

#### Aufgabe 1.3
Hier fehlt einiges an Information, nicht einmal die Abfahrten finden wir. Die finden wir in den `stop_times`. Fragen Sie dort doch mal den trip 1003122256 ab.

#### Aufgabe 1.4
Unbestreitbar spannendere Informationen. Anstatt nun in der letzten Tabelle, der `stops` den Namen der Bahnhöfe aufzulösen wollen wir nun eine View `fahrplan` erstellen, die uns den Fahrplan möglichst übersichtlich präsentiert. Erstellen Sie nun einen Join, über alle Tabellen und geben Sie folgende Informationen zurück:

- Name der Route konkateniert mit der anzeige des Trips
- ID des Trips
- Ankunftszeit am Halt
- Abfahrtszeit
- Der wievielte Halt auf der Strecke ist das (sequence)
- Name des Halts

Sortieren Sie nach trip_id und stop_sequence.

## Aufgabe 2

Betrachten Sie nun die im folgenden angegebene Zielstruktur:
![](https://nextcloud.th-deg.de/s/Kc7m486Z3KbZyB4/download?path=%2F&files=bahn-er.png)

Erstellen Sie eine View `Bahnhof` die wie abgebildet die beiden Attribute BahnhofId und Name hat.

# Fensterfunktionen

Fensterfunktionen sind im SQL:2003 Standard definiert. Diese Funktionen beschreiben eine Form der Aggregation die mit einem einfachen `GROUP BY` nicht erreicht werden kann: Es werden zwar die Werte mehrerer Datensätze aggregiert, also auf einen Wert gebracht, ohne aber dabei alle weiteren Elemente des Ergebnisses zu zwingen das Selbe zu tun. Grafisch stellt sich das wie folgt dar:

![Fensterfunktionen](https://www.sqlitetutorial.net/wp-content/uploads/2018/11/SQLite-window-function-vs-aggregate-function.png)

Quelle: [https://www.sqlitetutorial.net](https://www.sqlitetutorial.net/sqlite-window-functions/)

Links zwingt das Aggregat alle weiteren Werte dazu ebenfalls aggregiert zu werden. Rechts ist das Ergebnis des Aggregats einfach ein Teil jedes Ergebnistupels.



Die gegebenen Daten sind offensichtlich nicht in dem Format, das wir brauchen würden, um alleine die Relation Zug zu erstellen. Alle Informationen sind zwar in `fahrplan` enthalten, nur ist hier jede Reihe ein Halt eines Zuges. Um Start- und Endbahnhof zu bestimmen müsste man nach `trip_id` gruppieren und in die erste und letzte Zeile eines Zuges schauen könnten. Bis zur Standardtisierung der sog. Fensterfunktionen (window) waren dafür eine große Menge von Subselects nötig. Nun geht das deutlich einfacher.

Wir beschränken uns hier auf die Elemente, die wir zum Erreichen unseres Ziels brauchen. Der Aufbau einer Fensterfunktion ist wie folgt:

```
<AGGREGATFUNKTION> OVER([<PARTITION BY>] [<ORDER BY>] [<FRAMESPEC>])
```

- Eine Aggregatfunktion ist jede normale Aggregatfunkion, die wir im `GROUP BY` verwenden. Es gibt aber für Fenster eben auch neue Funktionen, z.b. `first_value(attribut), last_value(attribut)` oder zum Durchnumerieren `row_number()`. Eine Liste für SQLite finden Sie [hier](https://www.sqlite.org/windowfunctions.html#builtins).
- `OVER()` leitet ein Fenster ein. Spezifizieren Sie zwischen den Klammern keine Partition, keine Sortierung und keinen Frame ist es ein Fenster über alle Daten
- `PARTITION BY` gruppiert nach dem gegebenen Attribut
- `ORDER BY` sortiert die Liste, ohne dass das Auswirkungen auf die Sortierung der Query hat
- Die Framespec erlaubt Ihnen anzugeben wie groß Ihr Fenster sein soll. Das sprengt hier den Rahmen


## Aufgabe 3

Bringen wir nun also die Zug Relation in Form. Dafür brauchen wir eine Tabelle mit `zugnr, name, startbhf, endbhf`. Erstellen Sie also eine View anhand der Hinweise im Folgenden: 

Hinweis: kopieren Sie diese Zelle und entwickeln Sie hier Ihre Query
```
%sql
SELECT 
    DISTINCT trip_id as zugnr,
    as name -- TODO Hier erstellen wir eine Partition anhander der TRIP ID. Von dieser Partition nehmen wir den ersten Wert des namen
    as startbhf -- TODO Hier erstellen wir eine Partition anhander der TRIP ID. Von dieser Partition nehmen wir den ersten Wert der bahnhofs_id
    as endbhf -- TODO Hier erstellen wir eine Partition anhander der TRIP ID. Von dieser Partition nehmen wir den letzten Wert der bahnhofs_id
FROM fahrplan
ORDER BY RANDOM()
LIMIT 5;
```

# Aufgabe 4

Abschließend folgt noch die finale View. Jeder Abschnitt soll einzeln aufgelistet werden. Dazu brauchen wir die Zugnummer, die ID des *von* Bahnhofs, die ID des *zu* Bahnhofs, Abfahrts und Ankunftszeit. 

Das Vorgehen ist ähnlich wie oben, nur das wir für den nächsten Halt ja in die nächste Ergebniszeile schauen wollen. Das macht die `lead()` funktion für uns. Versuchen wir es erst einmal in einem SELECT:

Hinweis: kopieren Sie diese Zelle und entwickeln Sie hier Ihre Query
```
%%sql

SELECT
    *,
    AS next_stop_name, -- stop_name Analog aufgabe 3 nur dass wir lead verwenden
    AS next_stop_id -- stop_id Analog aufgabe 3 nur dass wir lead verwenden
FROM
    fahrplan;
```